In [ ]:
import pyart
import wradlib as wrl
import pandas as pd
import tempfile
import os
import numpy as np
import matplotlib.pyplot as plt

import pytz
import datetime as dt

from copy import deepcopy
import cartopy.crs as ccrs
import cartopy.io.shapereader as shpreader
import cartopy.feature as cfeature

import boto3
from botocore.handlers import disable_signing

import warnings
warnings.filterwarnings("ignore")

import sys

In [ ]:
#datestr = '2013/04/17'
datestr = sys.argv[1]

path_radarfiles = '/lcrc/group/earthscience/icrisologo/SAVEUR/gridded/'+datestr
savepath = '/lcrc/group/earthscience/icrisologo/KLOT_accum/1hr'

rr_retrieval_methods = ['rainrate','rainrate_from_kdp','rainrate_from_attenuation','rainrate_z_RT','rainrate_z_MP','rainrate_z_WC']

In [ ]:
# In[3]:


flist = os.listdir(path_radarfiles)
flist.sort()

In [ ]:
# Group the files according to hour, and put them in a dictionary where the hours are the keys.

# In[4]:


# create an empty dictionary
fname_dict = {}
for i in range(1,25,1):
    fname_dict[i] = []
# fill in dictionary
for i in range(len(flist)):
    #fname = keys[i].rsplit('/',1)[-1].strip('.nc')
    dtime_utc = dt.datetime.strptime(flist[i],'radar_KLOT_%Y%m%d_%H%M%S_gridded.nc')
    fname_dict[dtime_utc.hour+1].append(flist[i])

In [ ]:
# In[5]:


# Get time intervals average every hour
timediff_ave = {}
for i in range(1,25,1):
    timediff_ave[i] = []
for hour in list(fname_dict):
    timelist = []
    timediff = []
    fnames_within_hour = fname_dict[hour]
    for fname in fnames_within_hour:
        dtime_utc = dt.datetime.strptime(fname,'radar_KLOT_%Y%m%d_%H%M%S_gridded.nc')
        timelist.append(dtime_utc)
    for tdiff in np.diff(timelist):
        timediff.append(np.diff(timelist)[0].seconds)
    timediff_ave[hour] = np.mean(timediff)

In [ ]:
# In[9]:


for hour in list(fname_dict):
    print(hour,end='')
    fnames_within_hour = fname_dict[hour]

    # get current hour
    dtime_hr = dt.datetime.strptime(fnames_within_hour[0],'radar_KLOT_%Y%m%d_%H%M%S_gridded.nc')+dt.timedelta(hours=1)

    # set up grid base for hourly accum
    grid = pyart.io.read_grid(os.path.join(path_radarfiles,fnames_within_hour[0]))
    grid_base = deepcopy(grid)
    grid_base.fields = {}
    grid_base.time['units'] = dt.datetime.strftime(dtime_hr,'%Y/%m/%d %H:00:00')

    # initiate hour accum array
    hour_accum = np.zeros((len(rr_retrieval_methods),300,300))

    for fname in fnames_within_hour:#[:1]:
        print('.',end='')

        # read to radar object using pyart
        grid = pyart.io.read_grid(os.path.join(path_radarfiles,fname))

        for i,method in enumerate(rr_retrieval_methods):

            rdepth = grid.fields['radar_rainfall_depth_'+method]
            hour_accum[i] += np.squeeze(np.ma.filled(rdepth['data'],0.0))

    for i,method in enumerate(rr_retrieval_methods):
        # add hourly depth to grid base object
        hourdepth_dict = {}
        hourdepth_dict['data'] = np.expand_dims(hour_accum[i],axis=0)
        hourdepth_dict['RR method'] = method
        hourdepth_dict['long_name'] = 'Hourly accumulation ' + method
        hourdepth_dict['units'] = 'mm / time interval'
        hourdepth_dict['standard_name'] = 'hourlyaccum_'+method
        hourdepth_dict['coordinates'] = 'elevation azimuth range'

        grid_base.add_field('hourlyaccum_'+method, hourdepth_dict, replace_existing=True)

    # write grid to file
    pyart.io.write_grid(os.path.join(savepath,'KLOT_'+dt.datetime.strftime(dtime_hr,'%Y%m%d_%H0000')+'_accum1HR.nc'), grid_base)

#     print('')

